In [ ]:
# Train and Test YoloV5

# Author: Fabrizio Serpi

# This notebook is aimed at training and testing YoloV5, a family of compound-scaled object detection models.
# Dataset used for the following test is the Self-Driving-Car-small from: https://public.roboflow.com/object-detection/self-driving-car.

# The notebook is organized as follows:

#    1) Settings and YoloV5 download and verification
#    2) Dataset Download and Partition (Train, Validation, Test)
#    3) Train YoloV5 from scratch, using the medium size architecture
#        3.1) Train, using train and validation sets
#        3.2) Test using the test set (yolo validation, task test) 
#    4) Train YoloV5 starting from yolo pretrained weights (on COCO dataset), and freezing backbone network (feature extraction layers) 
#        4.1) Train, using train and validation sets
#        4.2) Test using the test set (yolo validation, task test) 
#    4) Start from the model obtained from scratch, and fine tune parameters and model training with evolve option
#    5) Starting from YoloV5 model trained from scratch, evolve parameters for 300 cycles over self driving car dataset
#    6) Train YoloV5 using evolved parameters
#        6.1) Train, using train and validation sets
#        6.2) Test using the test set (yolo validation, task test)

In [ ]:
# IMPORTS
import torch
import os
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime
import logging
import ipynbname
import sys
from pathlib import Path
import random
import shutil
import math
from IPython.utils import io
import yaml
from skimage.io import imread, imsave
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Setup logging for Stream and File output
cell_label = '[## LOGGING SETUP] - '

notebook_name = ipynbname.name()


# Retrieve date for log file name
now = datetime.now()
# Date and Time format: dd-mm-YY--H:M:S
date_time = now.strftime("%d-%m-%Y--%H-%M-%S")

logger = logging.getLogger(notebook_name)
logger.setLevel(logging.DEBUG)

# Append logs to a file
logs_folder = "./logs"
if not os.path.exists(logs_folder):
    os.mkdir(logs_folder)


log_filename = "{logs_folder}/{notebook_name}_{date_time}.log".format(logs_folder=logs_folder, notebook_name=notebook_name, date_time=date_time)
fhandler = logging.FileHandler(filename=log_filename, mode='a')
file_formatter = logging.Formatter('[%(asctime)s] %(name)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
fhandler.setFormatter(file_formatter)
fhandler.setLevel(logging.DEBUG)
logger.addHandler(fhandler)


# Setup logs for output stream
consoleHandler = logging.StreamHandler(sys.stdout)
console_formatter = logging.Formatter('[%(asctime)s] %(name)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
consoleHandler.setFormatter(console_formatter)
consoleHandler.setLevel(logging.DEBUG)
logger.addHandler(consoleHandler)


logger.info(cell_label + "Logging Setup Completed. Log file: {}, Notebook: {}".format(log_filename, notebook_name))

In [ ]:
# GPU SETTINGS
cell_label = '[## GPU SETTINGS] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Display full output
InteractiveShell.ast_node_interactivity = "all"

# Set an Nvidia card if you have more than one
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Check which GPU card is set
logger.info(cell_label + "Selected Nvidia GPU: {}".format(str(torch.cuda.get_device_name())))


logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# FUNCTIONS DEFINITION
cell_label = '[## FUNCTIONS DEFINITION] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")


# Load and start Tensorboard on the logDir given
def startTensorboard(activate, logDir):
    if activate:
        %load_ext tensorboard
        %tensorboard --logdir {logDir}

        
# Show a random image from the passed folder
def showRandomImageFromFolder(detections_folder):
    detection_images = [os.path.join(detections_folder, img) for img in os.listdir(detections_folder)]
    random_detection_image = imread(random.choice(detection_images))
    plt.imshow(random_detection_image)
    plt.show()
    

    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Download YoloV5 from its official Github Repository
cell_label = '[## RETRIEVE_YoloV5] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")


yolov5_folder_name = './yolov5'
git_command = 'git clone https://github.com/ultralytics/yolov5'  

if not os.path.isdir(yolov5_folder_name) or (os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) == 0)): 
    
    logger.info(cell_label + "Download YoloV5: {}".format(git_command))
    # Download YoloV5
    !{git_command} | tee -a {log_filename}

    if os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) > 0):
        
        logger.info(cell_label + "Install YoloV5 requirements")
        # Install YoloV5 requirements
        %pip install pip install -qr ./yolov5/requirements.txt
        
else:
    logger.info(cell_label + "YoloV5 previously Downloaded")
    

logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Test YoloV5 installation and inference using a pretrained model
cell_label = '[## TEST_YoloV5_PreTrained] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")



# Let's retrieve current date and time in order to track folders
now = datetime.now()
# Date and Time format: dd-mm-YY--H:M:S
dt_string = now.strftime("%d-%m-%Y--%H-%M-%S")


img_test = './yolov5/data/images/bus.jpg'
pretrained_weights = "yolov5s.pt"
project_name = "yolov5_train_detection"
run_name = "test_pretrained_" + dt_string


if os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) > 0):
    
    logger.info(cell_label + "Test YoloV5 in detection with a standard image and a pretrained COCO model: {}".format(pretrained_weights))
    inference_pretrained_command = './yolov5/detect.py --weights {pretrained_weights} --source {image_source} --project {project_name} --name {run_name}'.format(pretrained_weights = pretrained_weights, 
                                                                                                                                                                        image_source = img_test,
                                                                                                                                                                        project_name = project_name,
                                                                                                                                                                        run_name = run_name)

    %run {inference_pretrained_command}
    
else:
    logger.info(cell_label + "YoloV5 folder not found")
    
    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Download Data for Training
#     Download Self-Driving-Car-small dataset from: https://public.roboflow.com/object-detection/self-driving-car
#     Download the dataset in your notebook folder
cell_label = '[## DOWNLOAD_DATA_YOLOV5] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")



#yolov5_data = "self_driving_car_large"
yolov5_data = "self_driving_car_small"
yolov5_data_folder = os.path.join(".", yolov5_data)

# Use your personal link to download the dataset
link_to_dataset = ""

if not os.path.isdir(yolov5_data_folder) or (os.path.isdir(yolov5_data_folder) and (len(os.listdir(yolov5_data_folder)) == 0)): 
    
    logger.info(cell_label + "Downloading Dataset...")
    
    !curl -L {link_to_dataset} > {yolov5_data}.zip | tee -a {log_filename}; 
    !unzip -q {yolov5_data}.zip -d {yolov5_data_folder} | tee -a {log_filename}; 
    !rm {yolov5_data}.zip | tee -a {log_filename};

else:
    logger.info(cell_label + "Dataset already downloaded")
    
    
# Show two levels of data directory
logger.info(cell_label + "Downloaded dataset")
!tree {yolov5_data_folder} -L 2 | tee -a {log_filename}
    

logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Partition Data for Training
#    We need three sets, training, validation, test
cell_label = '[## PREPARE_DATA_TRAINING_YOLOV5] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")


data__train_test_folder = "./data_train_test_small"

train_folder = os.path.join(data__train_test_folder, "train")
val_folder = os.path.join(data__train_test_folder, "val")
test_folder = os.path.join(data__train_test_folder, "test")


train_images_folder = os.path.join(train_folder, "images")
train_labels_folder = os.path.join(train_folder, "labels")
val_images_folder   = os.path.join(val_folder, "images")
val_labels_folder   = os.path.join(val_folder, "labels")
test_images_folder  = os.path.join(test_folder, "images")
test_labels_folder  = os.path.join(test_folder, "labels")


if not os.path.isdir(data__train_test_folder) or (os.path.isdir(data__train_test_folder) and (len(os.listdir(data__train_test_folder)) == 0)): 
    
    # Create folders for data partitions and labels
    logger.info(cell_label + "Create folders for images and labels partitions")
    
    os.mkdir(data__train_test_folder) # Data main folder
    os.mkdir(train_folder)
    os.mkdir(val_folder)
    os.mkdir(test_folder)
    
    os.mkdir(train_images_folder)
    os.mkdir(train_labels_folder)
    os.mkdir(val_images_folder)
    os.mkdir(val_labels_folder)
    os.mkdir(test_images_folder)
    os.mkdir(test_labels_folder)
    
    
    # Partition data into train, validation, test
    train_fraction = 0.7
    val_fraction   = 0.2
    test_fraction  = 0.1

    logger.info(cell_label + "Data partionion - train fraction = {}, val fraction = {}, test fraction = {}".format(train_fraction, val_fraction, test_fraction))

    images_folder_path = "./{}/export/images".format(yolov5_data)
    lables_folder_path = "./{}/export/labels".format(yolov5_data)


    #List all the images
    imagesList = [Path(img).stem for img in os.listdir(images_folder_path) if os.path.isfile(os.path.join(images_folder_path, img))]
    
    logger.info(cell_label + "Total number of images: {}".format(len(imagesList)))

    
    # In place shuffle of the images list
    random.shuffle(imagesList)

    # Data partition
    imageListLength = len(imagesList)
    train_upper_bound = math.ceil((imageListLength*train_fraction))
    val_upper_bound   = math.ceil((imageListLength*(train_fraction+val_fraction)))

    train_data = imagesList[0:train_upper_bound]
    val_data   = imagesList[train_upper_bound:val_upper_bound]
    test_data  = imagesList[val_upper_bound:imageListLength]

    logger.info(cell_label + "Data partition, number of images for train: {}, validation: {}, test: {}".format(len(train_data), len(val_data), len(test_data)))
    
    
    # Copy images and labels in their destination folder
    
    # Copy train files
    logger.info(cell_label + "Copy Train Data...")
    for element in tqdm(train_data):
        src_img_path = os.path.join(images_folder_path, element + ".jpg")
        dst_img_path = os.path.join(train_images_folder, element + ".jpg")
        _ = shutil.copy(src_img_path, dst_img_path)
        
        src_lb_path = os.path.join(lables_folder_path, element + ".txt")
        dst_lb_path = os.path.join(train_labels_folder, element + ".txt")
        _ = shutil.copy(src_lb_path, dst_lb_path)
        
        
    # Copy validation files
    logger.info(cell_label + "Copy Val Data...")
    for element in tqdm(val_data):
        src_img_path = os.path.join(images_folder_path, element + ".jpg")
        dst_img_path = os.path.join(val_images_folder, element + ".jpg")
        _ = shutil.copy(src_img_path, dst_img_path)
        
        src_lb_path = os.path.join(lables_folder_path, element + ".txt")
        dst_lb_path = os.path.join(val_labels_folder, element + ".txt")
        _ = shutil.copy(src_lb_path, dst_lb_path)
        
        
    # Copy test files
    logger.info(cell_label + "Copy Test Data...")
    for element in tqdm(test_data):
        src_img_path = os.path.join(images_folder_path, element + ".jpg")
        dst_img_path = os.path.join(test_images_folder, element + ".jpg")
        _ = shutil.copy(src_img_path, dst_img_path)
        
        src_lb_path = os.path.join(lables_folder_path, element + ".txt")
        dst_lb_path = os.path.join(test_labels_folder, element + ".txt")
        _ = shutil.copy(src_lb_path, dst_lb_path)
    
    
    # Show two levels of data partitioned directory
    !tree {yolov5_data_folder} -L 2 |& tee -a {log_filename}
    
    # Check how many files there are in each data folder
    number_train_images = len(os.listdir(train_images_folder))
    number_train_labels = len(os.listdir(train_labels_folder))
    number_val_images = len(os.listdir(val_images_folder))
    number_val_labels = len(os.listdir(val_labels_folder))
    number_test_images = len(os.listdir(test_images_folder))
    number_test_labels = len(os.listdir(test_labels_folder))
    
    logger.info(cell_label + "Train data partition created - Train images: {}, Train labels: {}".format(number_train_images, number_train_labels))
    logger.info(cell_label + "Val data partition created - Val images: {}, Val labels: {}".format(number_val_images, number_val_labels))
    logger.info(cell_label + "Test data partition created - Test images: {}, Test labels: {}".format(number_test_images, number_test_labels))
    
    
else:
    logger.info(cell_label + "Dataset partitions already exist")
    

logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Create file Yaml for Training
cell_label = '[## CREATE_YAML_FILE] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")


# NOTE: you can use path as parameter if you want to give data folder an additional part of the path, i.e., the main folder path to the partitions

data_yaml_content = {'path': '..',
                     'train': train_folder,
                     'val': val_folder,
                     'test': test_folder,
                     'nc': 11,
                     'names': ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']}

data_yaml_file = os.path.join(data__train_test_folder, "data.yaml")

if os.path.isdir(data__train_test_folder):
    
    logger.info(cell_label + "Writing file yaml: {}".format(data_yaml_file))
    with open(data_yaml_file, "w+") as file:
        yaml.dump(data_yaml_content, file, default_flow_style=False)

        
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Train YoloV5 from scratch
cell_label = '[## TRAIN_YoloV5_from_scratch] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")


# Train parameters
#    NOTE: in order to train the machine starting from random weights we need a value for --cfg and an empty string for --weights 
SIZE = 512 # Dataset with small images
BATCH_SIZE = 64
EPOCHS = 300
WORKERS = 4
CFG = "yolov5m.yaml"
MODEL = ''
DATASET_SIZE = "small"
PROJECT = "yolov5_train_detection"
DATE_TIME = datetime.now().strftime("%d-%m-%Y--%H-%M-%S")
RUN_NAME = f"{DATE_TIME}-Model_{MODEL}-size_{SIZE}-epochs_{EPOCHS}-batch_{BATCH_SIZE}-{DATASET_SIZE}"
DATA_YAML = data_yaml_file
CACHE = "disk"
FURTHER_OPTIONS = "--exist-ok"

logger.info(cell_label + "(1) Training Parameters - IMAGE SIZE: {SIZE}, BATCH_SIZE: {BATCH_SIZE}, EPOCHS: {EPOCHS}, WORKERS: {WORKERS}, CACHE: {CACHE}".format(SIZE=SIZE, BATCH_SIZE=BATCH_SIZE, EPOCHS=EPOCHS, WORKERS=WORKERS, CACHE=CACHE))
logger.info(cell_label + "(2) Training Parameters - CFG: {CFG}, MODEL: {MODEL}, DATA_YAML: {DATA_YAML}, PROJECT: {PROJECT}, RUN_NAME: {RUN_NAME}, FURTHER_OPTIONS: {FURTHER_OPTIONS}".format(CFG=CFG, 
                                                                                                                                                                                        MODEL=MODEL, 
                                                                                                                                                                                        DATA_YAML=DATA_YAML, 
                                                                                                                                                                                        PROJECT=PROJECT, 
                                                                                                                                                                                        RUN_NAME=RUN_NAME,
                                                                                                                                                                                        FURTHER_OPTIONS=FURTHER_OPTIONS))


train_from_scratch_command = "./yolov5/train.py --img {SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} --data {DATA_YAML} --weights '' --cfg {CFG} --workers {WORKERS} --project {PROJECT} --name {RUN_NAME} --cache {CACHE} {FURTHER_OPTIONS}".format(SIZE=SIZE,
                                                                                                                                                                                                                       BATCH_SIZE=BATCH_SIZE,
                                                                                                                                                                                                                       EPOCHS=EPOCHS,
                                                                                                                                                                                                                       DATA_YAML=DATA_YAML,
                                                                                                                                                                                                                       CFG=CFG,
                                                                                                                                                                                                                       WORKERS=WORKERS,
                                                                                                                                                                                                                       PROJECT=PROJECT,
                                                                                                                                                                                                                       RUN_NAME=RUN_NAME,
                                                                                                                                                                                                                       CACHE=CACHE,
                                                                                                                                                                                                                       FURTHER_OPTIONS=FURTHER_OPTIONS)

logger.info(cell_label + "train_from_scratch_command: {}".format(train_from_scratch_command))

if os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) > 0):
    %run {train_from_scratch_command}

    
    
    
    
DELETE_CACHE = False

if(CACHE == 'disk' and DELETE_CACHE):
    logger.info(cell_label + "Remove data cache from disk (Train and Val)")
    
    DELETE_CACHE_TRAIN = "find {} -name '*.npy' -type f -delete".format(train_images_folder)
    !{DELETE_CACHE_TRAIN} |& tee -a {log_filename}
    
    DELETE_CACHE_VAL = "find {} -name '*.npy' -type f -delete".format(val_images_folder)
    !{DELETE_CACHE_VAL} |& tee -a {log_filename}
    
else:
    logger.info(cell_label + "No Request for cache to be removed / No cache used")
    
    

    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Start Tensorboard and Check Training Results
start_ts = True
startTensorboard(start_ts, PROJECT)

In [ ]:
# Test model obtained from the Training from Scratch
cell_label = '[## TEST_Model_YoloV5_from_scratch] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Path to your test data folder or single image
TEST_IMAGES = test_images_folder
SIZE = 512 # Dataset with small images

#Put here the path to your best weights
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)
CONF = 0.6
IOU  = 0.45

TEST_NAME = "{MODEL}--TEST_DETECTION".format(MODEL=MODEL)
TEST_PROJECT = PROJECT



logger.info(cell_label + "(1) Detection with Test Data - Parameters: TEST_PROJECT: {TEST_PROJECT}, TEST_NAME: {TEST_NAME}".format(TEST_PROJECT=TEST_PROJECT, TEST_NAME=TEST_NAME))
logger.info(cell_label + "(2) Detection with Test Data - Parameters: MODEL_WEIGHTS: {MODEL_WEIGHTS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS))
logger.info(cell_label + "(3) Detection with Test Data - Parameters: CONF: {CONF}, IOU: {IOU}".format(CONF=CONF, IOU=IOU))





test_from_scratch_command = "./yolov5/detect.py --source {TEST_IMAGES} --weights {MODEL_WEIGHTS} --img {SIZE} --conf {CONF} --iou {IOU} --name {TEST_NAME} --project {TEST_PROJECT}".format(TEST_IMAGES=TEST_IMAGES,
                                                                                                                                                 MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                                 SIZE=SIZE,
                                                                                                                                                 CONF=CONF,
                                                                                                                                                 IOU=IOU,
                                                                                                                                                 TEST_NAME=TEST_NAME,
                                                                                                                                                 TEST_PROJECT=TEST_PROJECT)


logger.info(cell_label + "Test Detection in progress....")

# NOTE: if output suppression 1 doesn't work, launch command as 2
mode = 2

if(mode == 1):
    with io.capture_output() as captured:
        %run {test_from_scratch_command}
elif(mode == 2):
    test_from_scratch_command = "python " + test_from_scratch_command
    _ = !{test_from_scratch_command}
    
    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Show random images from Test Detection Results
cell_label = '[## Show_Images--TEST_Model_YoloV5_from_scratch] - '
%matplotlib widget

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")



# Put here your detection folder
detections_folder = ""
detections_folder_path = "./yolov5_train_detection/{}/".format(detections_folder)
logger.info(cell_label + "detections_folder = {}".format(detections_folder_path))
showRandomImageFromFolder(detections_folder_path)


logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Validate model obtained from the 'Training from Scratch' using TEST DATA
cell_label = '[## VAL_Model_YoloV5_from_scratch_TEST_DATA] - '


logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Path to your test data folder
TEST_IMAGES = data_yaml_file
TASK = "test"

SIZE = 512 # Dataset with small images
BATCH_SIZE = 64

#Put here the path to your best weights
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)

TEST_NAME = "{MODEL}_VAL_test_data".format(MODEL=MODEL)
TEST_PROJECT   = "yolov5_train_detection"

OTHER_OPTIONS = "--augment"



logger.info(cell_label + "(1) Validation Parameters - TEST_IMAGES: {TEST_IMAGES}, TASK: {TASK}, IMAGE SIZE: {SIZE}, BATCH SIZE: {BATCH_SIZE}".format(TEST_IMAGES=TEST_IMAGES,
                                                                                                                                                 TASK=TASK,
                                                                                                                                                 SIZE=SIZE,
                                                                                                                                                 BATCH_SIZE=BATCH_SIZE))


logger.info(cell_label + "(2) Validation Parameters - MODEL_WEIGHTS: {MODEL_WEIGHTS}, TEST_NAME: {TEST_NAME}, TEST_PROJECT: {TEST_PROJECT}, OTHER_OPTIONS: {OTHER_OPTIONS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS, 
                                                                                                                                                                                   TEST_NAME=TEST_NAME, 
                                                                                                                                                                                   TEST_PROJECT=TEST_PROJECT, 
                                                                                                                                                                                   OTHER_OPTIONS=OTHER_OPTIONS))




val_command_test_data = "./yolov5/val.py --batch {BATCH_SIZE} --img {SIZE} --data {TEST_IMAGES} --weights {MODEL_WEIGHTS} --task {TASK} --project {TEST_PROJECT} --name {TEST_NAME} {OTHER_OPTIONS}".format(BATCH_SIZE=BATCH_SIZE,
                                                                                                                                         SIZE=SIZE,
                                                                                                                                         TEST_IMAGES=TEST_IMAGES,
                                                                                                                                         MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                         TASK=TASK,
                                                                                                                                         TEST_PROJECT=TEST_PROJECT,
                                                                                                                                         TEST_NAME=TEST_NAME,
                                                                                                                                         OTHER_OPTIONS=OTHER_OPTIONS)
logger.info(cell_label + "Test Data Validation in progress....")
%run {val_command_test_data}





logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Train YoloV5 starting from pretrained weights (Transfer Learning)
cell_label = '[## TRAIN_YoloV5_from_pretrained_weights] - '


logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")




# Train parameters
#    NOTE: in this case we are going to start from a pretrained model (COCO) and freeze the model backbone (Transfer Learning)

# In yolov5mV6 Backbone has 10 layers, from 0 to 9
FREEZE = 10
SIZE = 512 # Dataset with small images
BATCH_SIZE = 64
EPOCHS = 150
WORKERS = 4
MODEL = "yolov5m.pt"
DATASET_SIZE = "small"
PROJECT = "yolov5_train_detection"
DATE_TIME = datetime.now().strftime("%d-%m-%Y--%H-%M-%S")
RUN_NAME = f"{DATE_TIME}-Model_{MODEL}-size_{SIZE}-epochs_{EPOCHS}-batch_{BATCH_SIZE}-{DATASET_SIZE}-PRETRAINED-freeze{FREEZE}"
DATA_YAML = data_yaml_file
CACHE = "disk"
FURTHER_OPTIONS = "--exist-ok"





logger.info(cell_label + "(1) Training Parameters - IMAGE SIZE: {SIZE}, BATCH_SIZE: {BATCH_SIZE}, EPOCHS: {EPOCHS}, WORKERS: {WORKERS}, CACHE: {CACHE}".format(SIZE=SIZE, BATCH_SIZE=BATCH_SIZE, EPOCHS=EPOCHS, WORKERS=WORKERS, CACHE=CACHE))
logger.info(cell_label + "(2) Training Parameters - FREEZE: {FREEZE}, MODEL: {MODEL}, DATA_YAML: {DATA_YAML}, PROJECT: {PROJECT}, RUN_NAME: {RUN_NAME}, FURTHER_OPTIONS: {FURTHER_OPTIONS}".format(FREEZE=FREEZE, 
                                                                                                                                                                                               MODEL=MODEL, 
                                                                                                                                                                                               DATA_YAML=DATA_YAML,
                                                                                                                                                                                               PROJECT=PROJECT,
                                                                                                                                                                                               RUN_NAME=RUN_NAME,
                                                                                                                                                                                               FURTHER_OPTIONS=FURTHER_OPTIONS))

            

            





        
        
        


train_from_pretrained_command = "./yolov5/train.py --img {SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} --data {DATA_YAML} --weights {MODEL} --freeze {FREEZE} --workers {WORKERS} --project {PROJECT} --name {RUN_NAME} --cache {CACHE} {FURTHER_OPTIONS}".format(SIZE=SIZE,
                                                                                                                                                                                                                                                                BATCH_SIZE=BATCH_SIZE,
                                                                                                                                                                                                                                                                EPOCHS=EPOCHS,
                                                                                                                                                                                                                                                                DATA_YAML=DATA_YAML,
                                                                                                                                                                                                                                                                MODEL=MODEL,
                                                                                                                                                                                                                                                                FREEZE=FREEZE,
                                                                                                                                                                                                                                                                WORKERS=WORKERS,
                                                                                                                                                                                                                                                                PROJECT=PROJECT,
                                                                                                                                                                                                                                                                RUN_NAME=RUN_NAME,
                                                                                                                                                                                                                                                                CACHE=CACHE,
                                                                                                                                                                                                                                                                FURTHER_OPTIONS=FURTHER_OPTIONS)

                                                                                                                                                                                                                                       
logger.info(cell_label + "train_from_pretrained_command (Backbone freeze): {}".format(train_from_pretrained_command))


if(os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) > 0)): 
    logger.info(cell_label + "Start Training....")
    %run {train_from_pretrained_command}

    
    
    
    

    
    
DELETE_CACHE = False

if(CACHE == 'disk' and DELETE_CACHE):
    logger.info(cell_label + "Remove data cache from disk (Train and Val)")
    
    DELETE_CACHE_TRAIN = "find {} -name '*.npy' -type f -delete".format(train_images_folder)
    !{DELETE_CACHE_TRAIN} |& tee -a {log_filename}
    
    DELETE_CACHE_VAL = "find {} -name '*.npy' -type f -delete".format(val_images_folder)
    !{DELETE_CACHE_VAL} |& tee -a {log_filename}
    
else:
    logger.info(cell_label + "No Request for cache to be removed / No cache used")                           
               


logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Start Tensorboard and Check Training Results
start_ts = True
startTensorboard(start_ts, PROJECT)

In [ ]:
# Test model obtained from the Training from pretrained weights
cell_label = '[## TEST_Model_YoloV5_from_pretrained_weights] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Path to your test data folder or single image
TEST_IMAGES = test_images_folder
SIZE = 512 # Dataset with small images

#Put here the path to your best weights
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)
CONF = 0.6
IOU  = 0.45

TEST_NAME = "{MODEL}--TEST_DETECTION".format(MODEL=MODEL)
TEST_PROJECT = PROJECT




logger.info(cell_label + "(1) Detection with Test Data - Parameters - TEST_PROJECT: {TEST_PROJECT}, TEST_NAME: {TEST_NAME}".format(TEST_PROJECT=TEST_PROJECT, TEST_NAME=TEST_NAME))
logger.info(cell_label + "(2) Detection with Test Data - Parameters -  MODEL_WEIGHTS: {MODEL_WEIGHTS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS))
logger.info(cell_label + "(3) Detection with Test Data - Parameters -  CONF: {CONF}, IOU: {IOU}".format(CONF=CONF, IOU=IOU))




test_from_pretrained_command = "./yolov5/detect.py --source {TEST_IMAGES} --weights {MODEL_WEIGHTS} --img {SIZE} --conf {CONF} --iou {IOU} --name {TEST_NAME} --project {TEST_PROJECT}".format(TEST_IMAGES=TEST_IMAGES,
                                                                                                                                                 MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                                 SIZE=SIZE,
                                                                                                                                                 CONF=CONF,
                                                                                                                                                 IOU=IOU,
                                                                                                                                                 TEST_NAME=TEST_NAME,
                                                                                                                                                 TEST_PROJECT=TEST_PROJECT)


logger.info(cell_label + "Test Detection in progress....")

# NOTE: if output suppression 1 doesn't work, launch command as 2
mode = 2

if(mode == 1):
    with io.capture_output() as captured:
        %run {test_from_pretrained_command}
elif(mode == 2):
    test_from_pretrained_command = "python {}".format(test_from_pretrained_command)
    _ = !{test_from_pretrained_command}




    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Show random images from Test Detection Results
cell_label = '[## Show_Images--TRAIN_YoloV5_from_pretrained_weights] - '
%matplotlib widget

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")



# Put here your detection folder
detections_folder = ""
detections_folder_path = "./yolov5_train_detection/{}".format(detections_folder)
logger.info(cell_label + "detections_folder = {}".format(detections_folder_path))
showRandomImageFromFolder(detections_folder_path)


logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Validate model obtained from the 'Training from pretrained weights' using TEST DATA
cell_label = '[## VAL_Model_YoloV5_from_pretrained_weights_TEST_DATA] - '


logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Path to your test data folder
TEST_IMAGES = data_yaml_file
TASK = "test"

SIZE = 512 # Dataset with small images
BATCH_SIZE = 64

#Put here the path to your best weights
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)

TEST_NAME = "{MODEL}_VAL_test_data".format(MODEL=MODEL)
TEST_PROJECT   = "yolov5_train_detection"

OTHER_OPTIONS = "--augment"



logger.info(cell_label + "(1) Validation Parameters - TEST_IMAGES: {TEST_IMAGES}, TASK: {TASK}, IMAGE SIZE: {SIZE}, BATCH SIZE: {BATCH_SIZE}".format(TEST_IMAGES=TEST_IMAGES,
                                                                                                                                                 TASK=TASK,
                                                                                                                                                 SIZE=SIZE,
                                                                                                                                                 BATCH_SIZE=BATCH_SIZE))


logger.info(cell_label + "(2) Validation Parameters - MODEL_WEIGHTS: {MODEL_WEIGHTS}, TEST_NAME: {TEST_NAME}, TEST_PROJECT: {TEST_PROJECT}, OTHER_OPTIONS: {OTHER_OPTIONS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS, 
                                                                                                                                                                                   TEST_NAME=TEST_NAME, 
                                                                                                                                                                                   TEST_PROJECT=TEST_PROJECT, 
                                                                                                                                                                                   OTHER_OPTIONS=OTHER_OPTIONS))




val_command_test_data = "./yolov5/val.py --batch {BATCH_SIZE} --img {SIZE} --data {TEST_IMAGES} --weights {MODEL_WEIGHTS} --task {TASK} --project {TEST_PROJECT} --name {TEST_NAME} {OTHER_OPTIONS}".format(BATCH_SIZE=BATCH_SIZE,
                                                                                                                                         SIZE=SIZE,
                                                                                                                                         TEST_IMAGES=TEST_IMAGES,
                                                                                                                                         MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                         TASK=TASK,
                                                                                                                                         TEST_PROJECT=TEST_PROJECT,
                                                                                                                                         TEST_NAME=TEST_NAME,
                                                                                                                                         OTHER_OPTIONS=OTHER_OPTIONS)
logger.info(cell_label + "Test Data Validation in progress....")
%run {val_command_test_data}





logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Train starting from the 'Train from Scratch' model using evolve in order to tune the model and have modified hyperparameters
cell_label = '[## FINE_TUNE_YoloV5_from_scratch_with_Evolve] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")




# Train parameters
#    NOTE: in order to train the machine starting from random weights we need a value for --cfg and an empty string for --weights 
SIZE = 512 # Dataset with small images
BATCH_SIZE = 48
EPOCHS = 10
WORKERS = 4
# Put here the weights of the model you want to fine tune with evolve
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)
DATASET_SIZE = "small"
PROJECT = "yolov5_train_detection"
EVOLVE = 300
RUN_NAME = "{MODEL}-EVOLVE_HYP_{EVOLVE}".format(MODEL=MODEL, EVOLVE=EVOLVE)
DATA_YAML = data_yaml_file
CACHE = "disk"
FURTHER_OPTIONS = "--exist-ok"

logger.info(cell_label + "(1) Training Parameters - IMAGE SIZE: {SIZE}, BATCH_SIZE: {BATCH_SIZE}, EPOCHS: {EPOCHS}, EVOLVE: {EVOLVE}, WORKERS: {WORKERS}, CACHE: {CACHE}".format(SIZE=SIZE, 
                                                                                                                                                                             BATCH_SIZE=BATCH_SIZE, 
                                                                                                                                                                             EPOCHS=EPOCHS, 
                                                                                                                                                                             EVOLVE=EVOLVE,
                                                                                                                                                                             WORKERS=WORKERS, 
                                                                                                                                                                             CACHE=CACHE))

logger.info(cell_label + "(2) Training Parameters - MODEL: {MODEL_WEIGHTS}, DATA_YAML: {DATA_YAML}, PROJECT: {PROJECT}, RUN_NAME: {RUN_NAME}, FURTHER_OPTIONS: {FURTHER_OPTIONS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS, 
                                                                                                                                                                             DATA_YAML=DATA_YAML, 
                                                                                                                                                                             PROJECT=PROJECT, 
                                                                                                                                                                             RUN_NAME=RUN_NAME,
                                                                                                                                                                             FURTHER_OPTIONS=FURTHER_OPTIONS))


evolve_from_scratch_command = "./yolov5/train.py --img {SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} --data {DATA_YAML} --weights {MODEL_WEIGHTS} --evolve {EVOLVE} --workers {WORKERS} --project {PROJECT} --name {RUN_NAME} --cache {CACHE} {FURTHER_OPTIONS}".format(SIZE=SIZE,
                                                                                                                                                                                                                       BATCH_SIZE=BATCH_SIZE,
                                                                                                                                                                                                                       EPOCHS=EPOCHS,
                                                                                                                                                                                                                       DATA_YAML=DATA_YAML,
                                                                                                                                                                                                                       EVOLVE=EVOLVE,
                                                                                                                                                                                                                       MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                                                                                                       WORKERS=WORKERS,
                                                                                                                                                                                                                       PROJECT=PROJECT,
                                                                                                                                                                                                                       RUN_NAME=RUN_NAME,
                                                                                                                                                                                                                       CACHE=CACHE,
                                                                                                                                                                                                                       FURTHER_OPTIONS=FURTHER_OPTIONS)

logger.info(cell_label + "evolve_from_scratch_command: {}".format(evolve_from_scratch_command))

if os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) > 0):
    %run {evolve_from_scratch_command}

    
    
    
    
DELETE_CACHE = False

if(CACHE == 'disk' and DELETE_CACHE):
    logger.info(cell_label + "Remove data cache from disk (Train and Val)")
    
    DELETE_CACHE_TRAIN = "find {} -name '*.npy' -type f -delete".format(train_images_folder)
    !{DELETE_CACHE_TRAIN} |& tee -a {log_filename}
    
    DELETE_CACHE_VAL = "find {} -name '*.npy' -type f -delete".format(val_images_folder)
    !{DELETE_CACHE_VAL} |& tee -a {log_filename}
    
else:
    logger.info(cell_label + "No Request for cache to be removed / No cache used")
    
    

    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Check what EVOLVE has produced
evolve_model_folder = os.path.join("./", PROJECT, RUN_NAME)

!tree {evolve_model_folder} -L 1 | tee -a {log_filename}

In [ ]:
# Train YoloV5 starting from previous pretrained weights on the chosen dataset using evolved hyperparameters
cell_label = '[## TRAIN_YoloV5_Use_Evolve_Parameters] - '


logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")




# Train parameters
#    NOTE: in this case we are going to start from the pretrained model on the chosen dataset (Self Driving Car)
#          We are going to start from the model we trained from scratch and over which we have evolved training parameters
#          With this in place, we are going to train Yolo over the dataset with the same partitioning for 150 epochs.

SIZE = 512 # Dataset with small images
BATCH_SIZE = 64
EPOCHS = 600
WORKERS = 4
# Puth here yoyr model folder name
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)
DATASET_SIZE = "small"
PROJECT = "yolov5_train_detection"
DATE_TIME = datetime.now().strftime("%d-%m-%Y--%H-%M-%S")
RUN_NAME = f"{DATE_TIME}-Model_HyperParam-size_{SIZE}-epochs_{EPOCHS}-batch_{BATCH_SIZE}-{DATASET_SIZE}-PRETRAINED-EVOLVED_params"
DATA_YAML = data_yaml_file
CACHE = "disk"
FURTHER_OPTIONS = "--exist-ok"

# Put here the pat to your hyperparameter file
HYPER_PARAM = "./yolov5_train_detection/06-12-2022--14-46-01-Model_-size_512-epochs_1-batch_64-small-EVOLVE_HYP_1/hyp_evolve.yaml"






logger.info(cell_label + "(1) Training Parameters - IMAGE SIZE: {SIZE}, BATCH_SIZE: {BATCH_SIZE}, EPOCHS: {EPOCHS}, WORKERS: {WORKERS}, CACHE: {CACHE}".format(SIZE=SIZE, BATCH_SIZE=BATCH_SIZE, EPOCHS=EPOCHS, WORKERS=WORKERS, CACHE=CACHE))
logger.info(cell_label + "(2) Training Parameters - MODEL: {MODEL_WEIGHTS}, HYPER_PARAM: {HYPER_PARAM}, DATA_YAML: {DATA_YAML}, PROJECT: {PROJECT}, RUN_NAME: {RUN_NAME}, FURTHER_OPTIONS: {FURTHER_OPTIONS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS, HYPER_PARAM=HYPER_PARAM, 
                                                                                                                                                                             DATA_YAML=DATA_YAML,
                                                                                                                                                                             PROJECT=PROJECT,
                                                                                                                                                                             RUN_NAME=RUN_NAME,
                                                                                                                                                                             FURTHER_OPTIONS=FURTHER_OPTIONS))

            

            





        
        
        


train_custom_hyp_command = "./yolov5/train.py --img {SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} --hyp {HYPER_PARAM} --data {DATA_YAML} --weights {MODEL_WEIGHTS} --workers {WORKERS} --project {PROJECT} --name {RUN_NAME} --cache {CACHE} {FURTHER_OPTIONS}".format(SIZE=SIZE,
                                                                                                                                                                                                                                                                BATCH_SIZE=BATCH_SIZE,
                                                                                                                                                                                                                                                                EPOCHS=EPOCHS,
                                                                                                                                                                                                                                                                HYPER_PARAM=HYPER_PARAM,
                                                                                                                                                                                                                                                                DATA_YAML=DATA_YAML,
                                                                                                                                                                                                                                                                MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                                                                                                                                                WORKERS=WORKERS,
                                                                                                                                                                                                                                                                PROJECT=PROJECT,
                                                                                                                                                                                                                                                                RUN_NAME=RUN_NAME,
                                                                                                                                                                                                                                                                CACHE=CACHE,
                                                                                                                                                                                                                                                                FURTHER_OPTIONS=FURTHER_OPTIONS)

                                                                                                                                                                                                                                       
logger.info(cell_label + "train_custom_hyp_command: {}".format(train_custom_hyp_command))


if(os.path.isdir(yolov5_folder_name) and (len(os.listdir(yolov5_folder_name)) > 0)): 
    logger.info(cell_label + "Start Training....")
    %run {train_custom_hyp_command}

    
    
    
    

    
    
DELETE_CACHE = False

if(CACHE == 'disk' and DELETE_CACHE):
    logger.info(cell_label + "Remove data cache from disk (Train and Val)")
    
    DELETE_CACHE_TRAIN = "find {} -name '*.npy' -type f -delete".format(train_images_folder)
    !{DELETE_CACHE_TRAIN} |& tee -a {log_filename}
    
    DELETE_CACHE_VAL = "find {} -name '*.npy' -type f -delete".format(val_images_folder)
    !{DELETE_CACHE_VAL} |& tee -a {log_filename}
    
else:
    logger.info(cell_label + "No Request for cache to be removed / No cache used")                           
               


logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Start Tensorboard and Check Training Results
start_ts = True
startTensorboard(start_ts, PROJECT)

In [ ]:
# Test model obtained from the Training from pretrained weights
cell_label = '[## TEST_Model_YoloV5_from_evolved_hyperparameters] - '

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Path to your test data folder or single image
TEST_IMAGES = test_images_folder
SIZE = 512 # Dataset with small images

#Put here the path to your best weights
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)
CONF = 0.6
IOU  = 0.45

TEST_NAME = "{MODEL}--TEST_DETECTION".format(MODEL=MODEL)
TEST_PROJECT = PROJECT




logger.info(cell_label + "(1) Detection with Test Data - Parameters - TEST_PROJECT: {TEST_PROJECT}, TEST_NAME: {TEST_NAME}".format(TEST_PROJECT=TEST_PROJECT, TEST_NAME=TEST_NAME))
logger.info(cell_label + "(2) Detection with Test Data - Parameters -  MODEL_WEIGHTS: {MODEL_WEIGHTS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS))
logger.info(cell_label + "(3) Detection with Test Data - Parameters -  CONF: {CONF}, IOU: {IOU}".format(CONF=CONF, IOU=IOU))




test_from_pretrained_command = "./yolov5/detect.py --source {TEST_IMAGES} --weights {MODEL_WEIGHTS} --img {SIZE} --conf {CONF} --iou {IOU} --name {TEST_NAME} --project {TEST_PROJECT}".format(TEST_IMAGES=TEST_IMAGES,
                                                                                                                                                 MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                                 SIZE=SIZE,
                                                                                                                                                 CONF=CONF,
                                                                                                                                                 IOU=IOU,
                                                                                                                                                 TEST_NAME=TEST_NAME,
                                                                                                                                                 TEST_PROJECT=TEST_PROJECT)


logger.info(cell_label + "Test Detection in progress....")

# NOTE: if output suppression 1 doesn't work, launch command as 2
mode = 2

if(mode == 1):
    with io.capture_output() as captured:
        %run {test_from_pretrained_command}
elif(mode == 2):
    test_from_pretrained_command = "python {}".format(test_from_pretrained_command)
    _ = !{test_from_pretrained_command}




    
logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Show random images from Test Detection Results
cell_label = '[## Show_Images--TRAIN_YoloV5_from_evolved_hyperparameters] - '
%matplotlib widget

logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")



# Put here your detection folder
detections_folder = ""
detections_folder_path = "./yolov5_train_detection/{}".format(detections_folder)
logger.info(cell_label + "detections_folder = {}".format(detections_folder_path))
showRandomImageFromFolder(detections_folder_path)


logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")

In [ ]:
# Validate model obtained from the 'Training from pretrained weights and evolved hyperparameters' using TEST DATA
cell_label = '[## VAL_Model_YoloV5_from_evolved_hyperparameters_TEST_DATA] - '


logger.info("[------------------------------------------------------------------------------------]")
logger.info(cell_label + "START")

# Path to your test data folder
TEST_IMAGES = data_yaml_file
TASK = "test"

SIZE = 512 # Dataset with small images
BATCH_SIZE = 64

#Put here the path to your best weights
MODEL = ""
MODEL_WEIGHTS = "./yolov5_train_detection/{MODEL}/weights/best.pt".format(MODEL=MODEL)

TEST_NAME = "{MODEL}_VAL_test_data".format(MODEL=MODEL)
TEST_PROJECT   = "yolov5_train_detection"

OTHER_OPTIONS = "--augment"



logger.info(cell_label + "(1) Validation Parameters - TEST_IMAGES: {TEST_IMAGES}, TASK: {TASK}, IMAGE SIZE: {SIZE}, BATCH SIZE: {BATCH_SIZE}".format(TEST_IMAGES=TEST_IMAGES,
                                                                                                                                                 TASK=TASK,
                                                                                                                                                 SIZE=SIZE,
                                                                                                                                                 BATCH_SIZE=BATCH_SIZE))


logger.info(cell_label + "(2) Validation Parameters - MODEL_WEIGHTS: {MODEL_WEIGHTS}, TEST_NAME: {TEST_NAME}, TEST_PROJECT: {TEST_PROJECT}, OTHER_OPTIONS: {OTHER_OPTIONS}".format(MODEL_WEIGHTS=MODEL_WEIGHTS, 
                                                                                                                                                                                   TEST_NAME=TEST_NAME, 
                                                                                                                                                                                   TEST_PROJECT=TEST_PROJECT, 
                                                                                                                                                                                   OTHER_OPTIONS=OTHER_OPTIONS))




val_command_test_data = "./yolov5/val.py --batch {BATCH_SIZE} --img {SIZE} --data {TEST_IMAGES} --weights {MODEL_WEIGHTS} --task {TASK} --project {TEST_PROJECT} --name {TEST_NAME} {OTHER_OPTIONS}".format(BATCH_SIZE=BATCH_SIZE,
                                                                                                                                         SIZE=SIZE,
                                                                                                                                         TEST_IMAGES=TEST_IMAGES,
                                                                                                                                         MODEL_WEIGHTS=MODEL_WEIGHTS,
                                                                                                                                         TASK=TASK,
                                                                                                                                         TEST_PROJECT=TEST_PROJECT,
                                                                                                                                         TEST_NAME=TEST_NAME,
                                                                                                                                         OTHER_OPTIONS=OTHER_OPTIONS)
logger.info(cell_label + "Test Data Validation in progress....")
%run {val_command_test_data}





logger.info(cell_label + "FINISH")
logger.info("[------------------------------------------------------------------------------------]\n")